# Programming Assignment: Linear Regression 

In this assignment we will work with Ridge Regression, which uses a quadratic or L2 regularization. The model will predict possible earning while analyzing texts.
Let's take a look at original four tasks presented in Russian.

**Tasks will be duplicated in english further in this assignment**

1.  Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv (либо его заархивированную версию salary-train.zip).
2. Проведите предобработку:

    * Приведите тексты к нижнему регистру (text.lower()).
    * Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты:

    * Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).
    * Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'. Код для этого был приведен выше.
    * Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.
    * Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

3.  Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.

4.  Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

In [1]:
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack

import pandas

## Task №1
*Ru*: Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv (либо его заархивированную версию salary-train.zip)

*En:* Download data that contains job descriptions and corresponding annual salaries from the salary-train.csv file.

In [2]:
df = pandas.read_csv('salary-train.csv')
df

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355
...,...,...,...,...
59995,"As a result of continued growth, First Class S...",Whitley Bay,contract,26400
59996,PHP / MVC Web Developer MacclesfieldCirca ***...,Macclesfield,permanent,26000
59997,"Staff Nurse, Nursing Home, Baldock White Recru...",Baldock,NaN,24500
59998,This is one of the best agency side opportunit...,The City,permanent,65000


## Task №2
*Ru*: Проведите предобработку данных

*En:*Transform the data 

Приведите тексты к нижнему регистру (text.lower())  
Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты.  
Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'. 

Make texts lowercase (text.lower ())  
Replace everything except letters and numbers with spaces - this will make it easier to further divide the text into words. To do that following code could be used: re.sub ('[^ a-zA-Z0-9]', '', text). You can also use the replace method of the DataFrame to transform all texts at once.
Replace the whitespace in the LocationNormalized and ContractTime columns with the special string 'nan'. 

In [3]:
def data_preparation(df):
    column_list = ['FullDescription', 'LocationNormalized', 'ContractTime']
    for column in column_list:
        df[column] = df[column].map(lambda x: x.lower() if pandas.notnull(x) else x)
        df[column] = df[column].replace('[^a-zA-Z0-9]', ' ', regex = True)
        
    df['LocationNormalized'].fillna('nan', inplace=True)
    df['ContractTime'].fillna('nan', inplace=True)
    return df

In [4]:
df = data_preparation(df)
df

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,london,permanent,33000
1,an ideal opportunity for an individual that ha...,london,permanent,50000
2,online content and brand manager luxury reta...,south east london,permanent,40000
3,a great local marketleader is seeking a perman...,dereham,permanent,22500
4,registered nurse rgn nursing home for young...,sutton coldfield,nan,20355
...,...,...,...,...
59995,as a result of continued growth first class s...,whitley bay,contract,26400
59996,php mvc web developer macclesfieldcirca ...,macclesfield,permanent,26000
59997,staff nurse nursing home baldock white recru...,baldock,nan,24500
59998,this is one of the best agency side opportunit...,the city,permanent,65000


Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

Use TfidfVectorizer to convert texts to feature vectors. Leave only those words that occur in at least 5 objects (the min_df parameter of TfidfVectorizer). 

In [5]:
vectorizer = TfidfVectorizer(min_df = 5) #,lowercase= True 
X_train = vectorizer.fit_transform(df['FullDescription'])
X_train

<60000x22861 sparse matrix of type '<class 'numpy.float64'>'
	with 8365759 stored elements in Compressed Sparse Row format>

Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

Apply DictVectorizer to get one-hot coding of LocationNormalized and ContractTime rows. 

In [6]:
enc = DictVectorizer()
X_train_categ = enc.fit_transform(df[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_train_categ 

<60000x1766 sparse matrix of type '<class 'numpy.float64'>'
	with 120000 stored elements in Compressed Sparse Row format>

Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

Combine all the received characteristics into one matrix "objects-characteristics". Note that matrices for texts and categorical features are sparse. To combine their columns, use the scipy.sparse.hstack function. 

In [7]:
X_train = hstack([X_train, X_train_categ])

## Task №3
*Ru*: Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.

*En:* Train ridge regression with parameters alpha = 1 and random_state = 241. The target variable is written in the SalaryNormalized column 

In [8]:
y_train = df['SalaryNormalized']
clf = Ridge(alpha=1.0, random_state=241)
clf.fit(X_train, y_train)

Ridge(random_state=241)

## Task №4
*Ru*: Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

*En:* Make a predictions for two examples from the salary-test-mini.csv file. The values that you will get from forecasts are the answer to the task. 

In [9]:
test_df = pandas.read_csv('salary-test-mini.csv')
test_df

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [10]:
test_df = data_preparation(test_df)

X_test = vectorizer.transform(test_df['FullDescription'])
X_test_categ  = enc.transform(test_df[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test = hstack([X_test, X_test_categ])

clf.predict(X_test)

array([56579.70406425, 37132.68920761])